# Loading packages and libraries

In [ ]:
%pip install tensorflow_text

In [ ]:
%pip install tensorflow_addons

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import tensorflow_addons as tfa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [ ]:
tf.device('/device:GPU:2')

In [ ]:
data_df = pd.read_csv('label_raw.csv')

In [ ]:
data_df.head()

,Title,IsEnglish,SpecCat,ExpPeriod,Salary,Age,Gender,City,WorkType,WorkSchedule,...,EduLevel,N_Unis,LastUni,LastUni2,Top 10 Uni,URL,lastUniFaculty,lastUni2Faculty,DataAnalyst,DataScientist
0,BI аналитик,0.0,Продуктовый аналитик\nМаркетолог-аналитик\nАна...,32.0,NaN,25.0,0.0,Москва,1.0,0.0,...,Высшее образование (Магистр),5.0,"Московский гуманитарно-экономический институт,...",Российский экономический университет им. Г.В. ...,0.0,https://hh.ru/resume/83da3b970003d5d5870039ed1...,"аспирантура, Экономика и управление народным х...","МШБиМЭ, Международные Корпоративные Финансы (н...",1.0,0.0
1,Программист-разработчик,0.0,"Программист, разработчик\n",15.0,NaN,21.0,0.0,Москва,1.0,0.0,...,Высшее образование (Бакалавр),1.0,"Челябинский государственный университет, Челяб...",,0.0,https://hh.ru/resume/8606f37b000773de980039ed1...,"ИИТ, Программная инженерия",NaN,1.0,0.0
2,"Ведущий аналитик, BI-аналитик, Аналитик данных...",0.0,"Программист, разработчик\nBI-аналитик, аналити...",78.0,NaN,38.0,0.0,Пенза,1.0,0.0,...,Высшее образование,1.0,Пензенский Государственный Университет,,0.0,https://hh.ru/resume/e142604300013aa98d0039ed1...,"Факультет вычислительной техники, Специалист п...",NaN,1.0,1.0
3,Разработчик BI,0.0,"Программист, разработчик\n",100.0,170000.0,29.0,1.0,Москва,1.0,0.0,...,Высшее образование (Магистр),2.0,Warsaw School of Economics (Master Degree),Warsaw School of Economics (Bachelor Degree),0.0,https://hh.ru/resume/cb3d9ca600020eb8d00039ed1...,Advanced analytics Big Data,"Finance and accounting, Banking",1.0,0.0
4,Аналитик,0.0,Аналитик\n,30.0,160000.0,24.0,0.0,Москва,1.0,0.0,...,Высшее образование (Магистр),3.0,Гжельский государственный художественно-промыш...,Российская академия народного хозяйства и госу...,0.0,https://hh.ru/resume/ee2448210005c2f0080039ed1...,"Экономика, Экономика",Экономический факультет,1.0,0.0


# Undersampling

In [ ]:
df_da_1 = data_df[data_df['DataAnalyst']==1]
df_da_0 = data_df[data_df['DataAnalyst']==0]
df_da_0 = df_da_0.sample(len(df_da_1))

df_da_undersampling = df_da_0.append(df_da_1)
df_da_undersampling = df_da_undersampling.sample(frac=1).reset_index(drop=True)
df_da_undersampling = df_da_undersampling.drop(['DataScientist'], axis=1)

X_da_undersampling, y_da_undersampling = df_da_undersampling.drop(['DataAnalyst'], axis=1), df_da_undersampling['DataAnalyst']

<ipython-input-80-6138ae8f4e55>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_da_undersampling = df_da_0.append(df_da_1)


In [ ]:
df_ds_1 = data_df[data_df['DataScientist']==1]
df_ds_0 = data_df[data_df['DataScientist']==0]
df_ds_0 = df_ds_0.sample(len(df_ds_1))

df_ds_undersampling = df_ds_0.append(df_ds_1)
df_ds_undersampling = df_ds_undersampling.sample(frac=1).reset_index(drop=True)
df_ds_undersampling = df_ds_undersampling.drop(['DataAnalyst'], axis=1)

X_ds_undersampling, y_ds_undersampling = df_ds_undersampling.drop(['DataScientist'], axis=1), df_ds_undersampling['DataScientist']

<ipython-input-81-c00c638ca64e>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ds_undersampling = df_ds_0.append(df_ds_1)


In [ ]:
X_train_da, X_test_da, y_train_da, y_test_da = train_test_split(X_da_undersampling, y_da_undersampling, test_size=0.33)

In [ ]:
X_train_ds, X_test_ds, y_train_ds, y_test_ds = train_test_split(X_ds_undersampling, y_ds_undersampling, test_size=0.33)

# Oversampling

In [ ]:
X_da, y_da = data_df.drop(['DataAnalyst', 'DataScientist'], axis=1), data_df['DataAnalyst']
X_ds, y_ds = data_df.drop(['DataAnalyst', 'DataScientist'], axis=1), data_df['DataScientist']

# BERT feature extraction

In [ ]:
def concating_features(features, df):
  sen_w_feats = []
  df = df.fillna("")

  for (i, row) in df.iterrows():
    combined = ''

    for col_name in features:
      combined += (str(row[col_name]) + ' [SEP] ')
    
    sen_w_feats.append(combined)

  df['text'] = sen_w_feats
  df = df.drop(features, axis=1)
  return df

In [ ]:
text_features = ['Description', 'LastWorkDesc', 'LastWorkDesc2']

In [ ]:
X_text_ds_undersampling = X_ds_undersampling[text_features]

In [ ]:
X_text_ds_undersampling = concating_features(text_features, X_text_ds_undersampling)

In [ ]:
def Bert_model(X, y):
  bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
  bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessed_text = bert_preprocess(text_input)
  outputs = bert_encoder(preprocessed_text)

  l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
  l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

  model = tf.keras.Model(inputs=[text_input], outputs = [l])

  METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
  ]

  model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=METRICS)
  
  model.fit(X.text, y, epochs=10)

  return model

In [ ]:
model_text_ds = Bert_model(X_text_ds_undersampling, y_ds_undersampling)

Epoch 1/10
7/7 [==============================] - 16s 828ms/step - loss: 0.7513 - accuracy: 0.4099 - precision: 0.4153 - recall: 0.4414
Epoch 2/10
7/7 [==============================] - 4s 478ms/step - loss: 0.7286 - accuracy: 0.5045 - precision: 0.5053 - recall: 0.4324
Epoch 3/10
7/7 [==============================] - 3s 453ms/step - loss: 0.7120 - accuracy: 0.4775 - precision: 0.4699 - recall: 0.3514
Epoch 4/10
7/7 [==============================] - 3s 472ms/step - loss: 0.7033 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.6577
Epoch 5/10
7/7 [==============================] - 4s 499ms/step - loss: 0.7049 - accuracy: 0.5360 - precision: 0.5400 - recall: 0.4865
Epoch 6/10
7/7 [==============================] - 3s 453ms/step - loss: 0.6956 - accuracy: 0.5270 - precision: 0.5326 - recall: 0.4414
Epoch 7/10
7/7 [==============================] - 3s 452ms/step - loss: 0.7051 - accuracy: 0.4775 - precision: 0.4752 - recall: 0.4324
Epoch 8/10
7/7 [==============================] - 4s 5

In [ ]:
import joblib

filename = "bert_text_ds_model.joblib"
joblib.dump(model_text_ds, filename)

['bert_text_ds_model.joblib']

# Test data

In [ ]:
test_data = data_df[text_features]
test_data = concating_features(text_features, test_data)


In [ ]:
test_data

,text
0,Academic IELTS 8.0 [SEP] Работа над проектом I...
1,Имеется опыт в создании Android-приложений на ...
2,Ключевые знания и навыки:\nКачественный анализ...
3,"Энергичность, быстрая обучаемость, открытость...."
4,ОТВЕЧАЮ ТОЛЬКО В МЕССЕНДЖЕРАХ ИЛИ НА ПОЧТУ!!!\...
...,...
9557,"Семейное положение: женат, есть дети \r\nЖелае..."
9558,"Коммуникабельность, стрессоустойчивость, работ..."
9559,"Коммуникабельная, гибкая, целеустремленная, ис..."
9560,- Знаю участки работы универсального коммерчес...


In [ ]:
y_predicted = model_text_ds.predict(test_data.text)
y_predicted = y_predicted.flatten()

299/299 [==============================] - 153s 509ms/step


In [ ]:
check_y = np.where(y_predicted > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print(accuracy_score(check_y, data_df['DataScientist']))

0.6729763647772432


In [ ]:
print(f1_score(check_y, data_df['DataScientist']))

0.03338485316846986


In [ ]:
print(precision_score(check_y, data_df['DataScientist']))
print(recall_score(check_y, data_df['DataScientist']))

0.4864864864864865
0.017285531370038413


In [ ]:
results = pd.read_csv('result.csv')
# results = pd.DataFrame()

In [ ]:
results['ds'] = y_predicted

In [ ]:
results.to_csv('result.csv', index=False)